In [1]:
#Combining both Machine Learning code and Database MySQL
#Removed all extra parts of code, to see them visit testingic.ipynb
import tkinter as tk
from tkinter import filedialog

#for scanning qr code
import numpy as np
import pyzbar.pyzbar as pyzbar
from pyzbar.pyzbar import decode
import cv2


#final ml code till now
#Loading used Libraries
import FaceToolKit as ftk
import matplotlib.pyplot as plt
import numpy as np 
#For Image Capturing after 5 seconds
import cv2
import time
%matplotlib inline

try:
    
    cap = cv2.VideoCapture(0)

    while True:
        _, frame = cap.read()
        decodedObjects = pyzbar.decode(frame)
        cv2.imshow("Frame", frame)    
        key = cv2.waitKey(1)
        for obj in decodedObjects:
            p = obj.data
            idnum= p.decode("utf-8")
            #print("Type: ", obj.type)
            print("Scanned Id of Student ", idnum)
            cv2.destroyAllWindows()

            username=input("Enter your name: ")
            #idnum=input("Enter your id: ")
            photo=print("Upload your photo: ")

            root = tk.Tk()
            root.withdraw()

            file_path = filedialog.askopenfilename()

            #LOgin-logout part of code
            from datetime import datetime

            now = datetime.now()

            a = now.strftime("%H:%M:%S")
            print("User Logged In ", a)
            #Entering data from python to MySQL database
            #Creating database and sending images from path to it

            import mysql.connector
            from mysql.connector import Error

            def convertToBinaryData(filename):
                # Convert digital data to binary format
                with open(filename, 'rb') as file:
                    binaryData = file.read()
                return binaryData

            #YOU HAVE TO MAKE DATABASE AND TABLE IN MYSQL WORKBENCH FIRST THEN ADD DATA IN TABLE

            def insertBLOB(emp_id, name, photo,date,login, status, logout):
                print("Inserting BLOB into studentinfo1 table")
                try:
                    connection = mysql.connector.connect(host='localhost', database='student', user='aveen', password='1234')

                    cursor = connection.cursor()
                    sql_insert_blob_query = """ INSERT INTO studentinfo1 (id, name, photo,date,login,status,logout) VALUES (%s,%s,%s,%s,%s,%s,%s)"""

                    empPicture = convertToBinaryData(photo)

                    # Convert data into tuple format
                    insert_blob_tuple = (emp_id, name, empPicture,date,login, status, logout)
                    result = cursor.execute(sql_insert_blob_query, insert_blob_tuple)
                    connection.commit()
                    print("Image and file inserted successfully as a BLOB into studentinfo1 table", result)

                except mysql.connector.Error as error:
                    print("Failed inserting BLOB data into MySQL table {}".format(error))

                finally:
                    if (connection.is_connected()):
                        cursor.close()
                        connection.close()
                        print("MySQL connection is closed")

            #Specify the threshold and size of the network input
            verification_threshhold = 0.8
            image_size = 160

            #Creating an object from a written class
            v = ftk.Verification()

            #Load a trained model and specify the input and output network tensors
            # Pre-load model for Verification
            v.load_model("./models/20180204-160909/")
            v.initial_input_output_tensors()


            #img capturing after 5 sec
            capture = cv2.VideoCapture(0)
            capture.set(3, 640)
            capture.set(4, 480)
            img_counter = 0
            frame_set = []
            start_time = time.time()

            while True:
                ret, frame = capture.read()
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                cv2.imshow('frame', gray)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                if time.time() - start_time >= 5: #<---- Check if 5 sec passed
                    img_name = "C:/Users/Dell/Desktop/images/frame{}.jpg".format(img_counter)
                    cv2.imwrite(img_name, frame)
                    start_time = time.time()
                    img1 = plt.imread(img_name) #set path to first image coming from webcam

                    img2 = plt.imread(file_path)#set path to second image coming from folder


                    #Resizing first image
                    import cv2
                    img = cv2.imread(img_name, cv2.IMREAD_UNCHANGED)
                    width = 160
                    height = 160
                    dim = (width, height) 
                    # resize image
                    img1 = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)


                    #Resizing second image
                    img = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
                    width = 160
                    height = 160
                    dim = (width, height) 
                    # resize image
                    img2 = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)


                    #embeddings for both images
                    emb1 = v.img_to_encoding(img1, image_size)
                    emb2 = v.img_to_encoding(img2, image_size)

                    #Formula for embeddings
                    def distance(emb1, emb2):
                        diff = np.subtract(emb1, emb2)
                        return np.sum(np.square(diff))
                    #distance
                    dist = distance(emb1, emb2)
                    is_same = dist < verification_threshhold

                    #For current date
                    from datetime import date
                    today = date.today()


                    if is_same==True:
                        c="User is Online"
                        print(c)
                        b=' '
                        insertBLOB(idnum, username,file_path,today,a,c,b)


                    else:
                        cv2.destroyAllWindows()
                        break


                    #Script that deletes each picture one by one
                    import glob,os
                    directory="C:/Users/Dell/Desktop/images"
                    for i in glob.glob(os.path.join(directory,"*.jpg")):
                        try:
                            os.chmod(i,0o777)
                            #os.remove(i)
                        except OSError:
                            pass
                img_counter += 1
                #img counter for next image

            #For logout time
            now = datetime.now()
            b = now.strftime("%H:%M:%S")
            print("User Logged Out ", b)
            #insertBLOB(idnum, username,file_path,today,a,c,b)

except:
    print("Exception")


C:\Users\Dell\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Dell\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Dell\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Dell\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passing (

Scanned Id of Student  456789
Enter your name: aveenii
Upload your photo: 
User Logged In  15:38:54
Model directory: ./models/20180204-160909/
Metagraph file: model-20180204-160909.meta
Checkpoint file: model-20180204-160909.ckpt-266000
'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./models/20180204-160909/model-20180204-160909.ckpt-266000
User is Online
Inserting BLOB into studentinfo1 table
Image and file inserted successfully as a BLOB into studentinfo1 table None
MySQL connection is closed
User is Online
Inserting BLOB into studentinfo1 table
Image and file inserted successfully as a BLOB into studentinfo1 table None
MySQL connection is closed
User is Online
Inserting BLOB into studentinfo1 table
Image and file inserted succ

In [1]:
#Combining both Machine Learning code and Database MySQL
#Removed all extra parts of code, to see them visit testingic.ipynb
import tkinter as tk
from tkinter import filedialog

#for scanning qr code
import numpy as np
import pyzbar.pyzbar as pyzbar
from pyzbar.pyzbar import decode
import cv2


#final ml code till now
#Loading used Libraries
import FaceToolKit as ftk
import matplotlib.pyplot as plt
import numpy as np 
#For Image Capturing after 5 seconds
import cv2
import time
%matplotlib inline
import mysql.connector



cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    decodedObjects = pyzbar.decode(frame)
    cv2.imshow("Frame", frame)    
    key = cv2.waitKey(1)
    for obj in decodedObjects:
        p = obj.data
        idnum= p.decode("utf-8")
        #print("Type: ", obj.type)
        print("Scanned Id of Student ", idnum)
        cv2.destroyAllWindows()
        #FOR CHECKING IF THE SCANNED ID EXISTS IN REGISTRATION TABLE
        mydb = mysql.connector.connect(host='localhost', database='student', user='aveen', password='1234')
        mycursor = mydb.cursor()
        sql = "SELECT * FROM registration  WHERE enrolment_num = %s"
        adr = (idnum, )

        mycursor.execute(sql, adr)

        myresult = mycursor.fetchall()

        for x in myresult:
              print("Student is Found in Registartion Table", x)
        

        username=input("Enter your name: ")
        #idnum=input("Enter your id: ")
        photo=print("Upload your photo: ")

        root = tk.Tk()
        root.withdraw()

        file_path = filedialog.askopenfilename()

        #LOgin-logout part of code
        from datetime import datetime

        now = datetime.now()

        a = now.strftime("%H:%M:%S")
        print("User Logged In ", a)
        #Entering data from python to MySQL database
        #Creating database and sending images from path to it

        import mysql.connector
        from mysql.connector import Error

        def convertToBinaryData(filename):
            # Convert digital data to binary format
            with open(filename, 'rb') as file:
                binaryData = file.read()
            return binaryData

        #YOU HAVE TO MAKE DATABASE AND TABLE IN MYSQL WORKBENCH FIRST THEN ADD DATA IN TABLE

        def insertBLOB(emp_id,login, face_recognition, logout,eye_tracking, screen_monitoring ,speech,enrolment_num ):
            print("Inserting BLOB into student_info table")
            try:
                connection = mysql.connector.connect(host='localhost', database='student', user='aveen', password='1234')

                cursor = connection.cursor()
                sql_insert_blob_query = """ INSERT INTO student_info (id,login,face_recognition,logout,eye_tracking, screen_monitoring ,speech,enrolment_num) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"""

                #empPicture = convertToBinaryData(photo)

                # Convert data into tuple format
                insert_blob_tuple = (emp_id,login, face_recognition, logout,eye_tracking, screen_monitoring ,speech,enrolment_num)
                result = cursor.execute(sql_insert_blob_query, insert_blob_tuple)
                connection.commit()
                print("Image and file inserted successfully as a BLOB into student_info table", result)

            except mysql.connector.Error as error:
                print("Failed inserting BLOB data into MySQL table {}".format(error))

            finally:
                if (connection.is_connected()):
                    cursor.close()
                    connection.close()
                    print("MySQL connection is closed")

        #Specify the threshold and size of the network input
        verification_threshhold = 0.8
        image_size = 160

        #Creating an object from a written class
        v = ftk.Verification()

        #Load a trained model and specify the input and output network tensors
        # Pre-load model for Verification
        v.load_model("./models/20180204-160909/")
        v.initial_input_output_tensors()


        #img capturing after 5 sec
        capture = cv2.VideoCapture(0)
        capture.set(3, 640)
        capture.set(4, 480)
        img_counter = 0
        frame_set = []
        start_time = time.time()

        while True:
            ret, frame = capture.read()
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            cv2.imshow('frame', gray)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            if time.time() - start_time >= 5: #<---- Check if 5 sec passed
                img_name = "C:/Users/Aveen Faheem/Desktop/Anaconda_work/deep-face-recognition-master/deep-face-recognition-master/img/frame{}.jpg".format(img_counter)
                cv2.imwrite(img_name, frame)
                start_time = time.time()
                img1 = plt.imread(img_name) #set path to first image coming from webcam

                img2 = plt.imread(file_path)#set path to second image coming from folder


                #Resizing first image
                import cv2
                img = cv2.imread(img_name, cv2.IMREAD_UNCHANGED)
                width = 160
                height = 160
                dim = (width, height) 
                # resize image
                img1 = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)


                #Resizing second image
                img = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
                width = 160
                height = 160
                dim = (width, height) 
                # resize image
                img2 = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)


                #embeddings for both images
                emb1 = v.img_to_encoding(img1, image_size)
                emb2 = v.img_to_encoding(img2, image_size)

                #Formula for embeddings
                def distance(emb1, emb2):
                    diff = np.subtract(emb1, emb2)
                    return np.sum(np.square(diff))
                #distance
                dist = distance(emb1, emb2)
                is_same = dist < verification_threshhold

                #For current date
                from datetime import date
                today = date.today()


                if is_same==True:
                    c="User is Online"
                    print(c)
                    b=' '
                    d=' '
                    e=' '
                    f=' '
                    id=1
                    insertBLOB(id,a,c,b,d,e,f,idnum)


                else:
                    cv2.destroyAllWindows()
                    break


                #Script that deletes each picture one by one
                import glob,os
                directory="C:/Users/Aveen Faheem/Desktop/Anaconda_work/deep-face-recognition-master/deep-face-recognition-master/img"
                for i in glob.glob(os.path.join(directory,"*.jpg")):
                    try:
                        os.chmod(i,0o777)
                        os.remove(i)
                    except OSError:
                        pass
            img_counter += 1
            #img counter for next image

        #For logout time
        now = datetime.now()
        b = now.strftime("%H:%M:%S")
        print("User Logged Out ", b)
        insertBLOB(id,a,c,b,d,e,f,idnum)


Scanned Id of Student  NED/1001/2019-20
Student is Found in Registartion Table (1, 'NED/1001/2019-20', 'sidra ', 'a', 2019, 'software engineering', 'http://studentcorner.rcai.pk/imgg/sir.jpg')
Enter your name: Aveen
Upload your photo: 
User Logged In  00:45:26
Model directory: ./models/20180204-160909/
Metagraph file: model-20180204-160909.meta
Checkpoint file: model-20180204-160909.ckpt-266000
'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./models/20180204-160909/model-20180204-160909.ckpt-266000
User is Online
Inserting BLOB into student_info table
Image and file inserted successfully as a BLOB into student_info table None
MySQL connection is closed
User is Online
Inserting BLOB into student_info table
Image and file inserted 

TypeError: cannot unpack non-iterable NoneType object